In [1]:
source("../../BrusselSprouts/scripts/functions.R")
scripts_dir="/home/jnrunge/data/trd/mapped_reads/scripts/"
initial_timedate=Sys.time()
library(tidytable)


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


This can lead to most dplyr functions being overwritten by tidytable functions.


Attaching package: 'tidytable'


The following objects are masked from 'package:dplyr':

    across, add_count, add_tally, anti_join, arrange, between,
    bind_cols, bind_rows, c_across, case_match, case_when, coalesce,
    consecutive_id, count, cross_join, cume_dist, cur_column, cur_data,
    cur_group_id, cur_group_rows, dense_rank, desc, distinct, filter,
    first, full_join, group_by, group_cols, group_split, group_vars,
    if_all, if_any, if_else, inner_join, is_grouped_df, lag, last,
    lead, left_join, min_rank, mutate, n, n_distinct, na_if, nest_by,
    nest_join, nth, percent_rank, pick, pull, recode, reloca

In [2]:
c="YJNRC18"

In [3]:
AS=fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz.allelesharing.csv.gz"))

In [7]:
TRD=fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz"))
head(TRD)

chr,pos,alleles,GT1,GT2,ADcross,DPcross,AD3_0,AD3_1,AD3_2,AD3_3,Allele1,Allele2,AD_A1,AD_A2,global_pos,sumCount,smoothed
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
chromosome1,13661,"A,G",1/1,0/0,"53,148",201,53,148,NA,NA,1,0,148,53,13661,201,0.4544824
chromosome1,27729,"C,A",0/0,1|1,"92,159",251,92,159,NA,NA,0,1,92,159,27729,251,0.4618821
chromosome1,27732,"G,T",0/0,1|1,"92,153",245,92,153,NA,NA,0,1,92,153,27732,245,0.4618840
chromosome1,27738,"A,C",0/0,1|1,"81,148",229,81,148,NA,NA,0,1,81,148,27738,229,0.4618878
chromosome1,27744,"T,C",0/0,1|1,"83,148",231,83,148,NA,NA,0,1,83,148,27744,231,0.4618917
chromosome1,27747,"T,G,A",0/0,2|2,"81,0,144",225,81,0,144,NA,0,2,81,144,27747,225,0.4618936


In [4]:
TRD_loci=fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-TRD_regions.csv.gz"))

In [6]:
head(AS)

#CHROM,POS,AAA,AAB,AAC,AAD,AAE,AAG,AAH,AAI,...,XTRA_FFI,XTRA_FFJ,XTRA_FFK,XTRA_FFL,XTRA_FFM,XTRA_FFN,XTRA_FFO,XTRA_FFP,XTRA_FFQ,chrpos
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chromosome1,13661,A2_hom,A2_hom,A2_hom,A2_hom,A2_hom,A2_hom,A2_hom,A2_hom,...,A2_hom,A2_hom,A2_hom,A2_hom,A2_hom,A2_hom,A2_hom,A2_hom,A2_hom,Other
chromosome1,27729,A2_hom,A2_hom,Other,Other,A2_hom,Other,Other,A2_hom,...,A2_hom,A2_hom,A2_hom,Other,Other,A2_hom,Other,A1_hom,A2_hom,Other
chromosome1,27732,A1_hom,A2_hom,Other,Other,A1_hom,Other,Other,A1_hom,...,A1_hom,A2_hom,A2_hom,Other,Other,A2_hom,Other,A1_hom,A2_hom,Other
chromosome1,27738,A2_hom,A2_hom,Other,Other,A2_hom,Other,Other,A2_hom,...,A2_hom,A2_hom,A2_hom,Other,Other,A2_hom,Other,A1_hom,A2_hom,Other
chromosome1,27744,A2_hom,A2_hom,Other,Other,A2_hom,Other,Other,A2_hom,...,A2_hom,A2_hom,A2_hom,Other,Other,A2_hom,Other,A1_hom,A1_hom,Other
chromosome1,27747,A2_hom,A2_hom,Other,Other,A2_hom,Other,Other,A2_hom,...,A2_hom,Other,Other,Other,Other,Other,Other,A1_hom,A1_hom,Other


In [9]:
TRD_loci

ID,lengthSNPs,chr_start,chr_end,global_start,global_end
<int>,<int>,<chr>,<chr>,<int>,<int>
2,615,chromosome4,chromosome4,1725459,1862052
4,1261,chromosome5,chromosome5,2872884,3193716
6,567,chromosome13,chromosome13,8440879,8605924


In [22]:
i=1
if(TRD_loci$chr_start[i]!=TRD_loci$chr_end[i]){
    stop("chr overlapping TRD")
}
TRD_subset=filter(TRD, chr== TRD_loci$chr_start[i] & global_pos >= TRD_loci$global_start[i] & global_pos <= TRD_loci$global_end[i])
df_AS_filtered=filter(AS, `#CHROM` == TRD_loci$chr_start[i], POS %in% TRD_subset$pos)
melted=reshape2::melt(df_AS_filtered, id.vars = c("#CHROM","POS"))
melted=filter(melted, variable != "chrpos")
tmp=summarise(group_by(melted, variable), nAll=n())
vcf_translated_summary=left_join(tmp,summarise(group_by(melted, variable, value), n=n()), by=c("variable"))%>%mutate(p=n/nAll)%>%select(variable,value,p)%>%rename(Strain=variable, Type=value)
  

In [24]:
arrange(vcf_translated_summary, -p)

Strain,Type,p
<fct>,<chr>,<dbl>
ACI,A1_hom,0.9967480
ADA,A2_hom,0.9967480
AKI,A2_hom,0.9967480
APQ,A2_hom,0.9967480
APR,A2_hom,0.9967480
ATE,A2_hom,0.9967480
ATP,A2_hom,0.9967480
SACE_YBP,A2_hom,0.9967480
SACE_YCV,A2_hom,0.9967480
